In [1]:
import pandas as pd
import pathlib
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder

In [2]:
path = pathlib.Path().cwd().parents[1] / 'CSVs' / 'Kings County Housing.csv'
df = pd.read_csv(path)
df.head()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,sqft_basement,yr_built,yr_renovated,zipcode
0,1,221900,3,1.00,1180,5650,1.0,0,0,0,1955,0,98178
1,2,538000,3,2.25,2570,7242,2.0,0,0,400,1951,1991,98125
2,3,180000,2,1.00,770,10000,1.0,0,0,0,1933,0,98028
3,4,604000,4,3.00,1960,5000,1.0,0,0,910,1965,0,98136
4,5,510000,3,2.00,1680,8080,1.0,0,0,0,1987,0,98074


- a) Where is King County? Use the zip codes if you are unsure.
    - It is in Washington

- b) How many observations are in the dataset? What does 1 row correspond to?
    - There are 21613 observations in the dataset
    - Each row corresponds to a house in the county

In [3]:
df.shape

(21613, 13)

- c) What are the median statistics for price, bedrooms, bathrooms, square foot of living space, and year built?
    - Price: 450000
    - Bedrooms: 3
    - Bathrooms: 2.25
    - Square Foot of Living Space: 1910
    - Year Built: 1975


In [4]:
df[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'yr_built']].describe()

,price,bedrooms,bathrooms,sqft_living,yr_built
count,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000
mean,5.400881e+05,3.370842,2.114757,2079.899736,1971.005136
std,3.671272e+05,0.930062,0.770163,918.440897,29.373411
min,7.500000e+04,0.000000,0.000000,290.000000,1900.000000
25%,3.219500e+05,3.000000,1.750000,1427.000000,1951.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,1975.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1997.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,2015.000000


- d) Run the regression: $$P rice = a + b ∗Bedrooms$$
    - Write a full sentence explaining the coefficient on bedrooms.
        - The coefficient on bedrooms is about $121716$ which means that every additional bedroom corresponds to an increase of house price by about $121716$ dollars.
    - Is the coefficient statistically significant? What is the 95% confidence interval on the coefficient on bedrooms? Interpret the interval.
        - The coefficient is statistically significant because the p-value is less than $0.05$.
        - The 95% confidence interval on the coefficient on bedrooms is $[1.17 * 10^5, 1.27 * 10^5]$ which means that we are $95\%$ confident that the true coefficient on bedrooms is between $117000$ and $127000$.
    - If a house has 2 bedrooms, what does the one variable model predict the price will be?
        - Approximately $\$373234.61$
    - Is the relationship between bedrooms and price necessarily causal?
        - The relationship between bedrooms and price is not necessarily causal. There could be other factors that affect the price of a house that are not accounted for in this model.
    - Interpert the $R^2$ value of this model.
        - The $R^2$ value of this model is $0.095$ which means that $9.5\%$ of the variation in price can be explained by the number of bedrooms.

In [5]:
X = df['bedrooms']
Y = df['price']

X_full = sm.add_constant(X)

model = sm.OLS(Y, X_full)
results = model.fit()

display(results.summary())
f'The equation of the line is: y = {results.params[0]:.2f} + {results.params[1]:.2f}x'

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     2271.
Date:                Tue, 21 Nov 2023   Prob (F-statistic):               0.00
Time:                        22:46:18   Log-Likelihood:            -3.0652e+05
No. Observations:               21613   AIC:                         6.131e+05
Df Residuals:                   21611   BIC:                         6.131e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.298e+05   8931.866     14.533      0.000    1.12e+05    1.47e+05
bedrooms    1.217e+05   2554.304     47.651      0.000    1.17e+05    1.27e+05
==============================================================================
Omnibus:                    18859.406   Durbin-Watson:                   1.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1199044.960
Skew:                           3.904   Prob(JB):                         0.00
Kurtosis:                      38.644   Cond. No.                         14.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

'The equation of the line is: y = 129802.36 + 121716.13x'

In [6]:
results.params[0] + results.params[1] * 2

373234.6093419398

- e) Run the regression of price on bedrooms and living square footage: $$\text{Price} = a + b * \text{Bedrooms} + c * \text{Sqft\_living}$$
    - Write a full sentence explaining the coefficient on bedrooms. How has it changed? Why might it have changed?
        - The coefficient of bedrooms is now negative (from $1.298*10^5 \text{ to } -5.707 * 10^4$), which corresponds to a decrease in the price of the house when the number of bedrooms increases. The coefficient might have changed because of the addition of a new variable or because more bedrooms could actually decrease the price of a house. In addition to this, there may be a different variable that increases the value of a house.
    - How has the $R^2$ changed from the first model?
        - The $R^2$ value has increased from $0.095$ to $0.507$ which means that 50.7% of the variability in housing prices can be explained by the number of bedrooms and the square footage of living space.
    - What does the model predict for the price of a 2 bedroom, 1000 square foot apartment?
        - 279,284.53
    - What does the model predict for the price of a 3 bedroom, 1000 square foot apartment?
        - 222,217.77


In [7]:
X = df[['bedrooms', 'sqft_living']]
Y = df['price']

X_full = sm.add_constant(X)

model = sm.OLS(Y, X_full)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                 1.110e+04
Date:                Tue, 21 Nov 2023   Prob (F-statistic):               0.00
Time:                        22:46:18   Log-Likelihood:            -2.9997e+05
No. Observations:               21613   AIC:                         5.999e+05
Df Residuals:                   21610   BIC:                         6.000e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        7.947e+04   6604.764     12.032      0.000    6.65e+04    9.24e+04
bedrooms    -5.707e+04   2308.223    -24.723      0.000   -6.16e+04   -5.25e+04
sqft_living   313.9487      2.337    134.314      0.000     309.367     318.530
==============================================================================
Omnibus:                    14423.033   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           492253.321
Skew:                           2.732   Prob(JB):                         0.00
Kurtosis:                      25.732   Cond. No.                     8.87e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.87e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
results.params

const          79469.359075
bedrooms      -57066.758923
sqft_living      313.948686
dtype: float64

In [9]:
print(f'What does the model predict for the price of a 2 bedroom, 1000 square foot apartment?\n{results.params[0] + results.params[1] * 2 + results.params[2] * 1000:,.2f}')
print(f'What does the model predict for the price of a 3 bedroom, 1000 square foot apartment?\n{results.params[0] + results.params[1] * 3 + results.params[2] * 1000:,.2f}')

What does the model predict for the price of a 2 bedroom, 1000 square foot apartment?
279,284.53
What does the model predict for the price of a 3 bedroom, 1000 square foot apartment?
222,217.77


- f) Add dummies for zip code to your second model and run the regression: $$\text{Price} = a + b * \text{Bedrooms} + c * \text{Sqft\_living} + d * \text{Zip}$$ You should have 70 zip dummies. You do not need to interpret them, just include them.
    - What is the $R^2$ of this model? Write a full sentence.
        - The $R^2$ value of this model is $0.738$ which means that $73.8\%$ of the variation in price can be explained by the number of bedrooms, square footage of living space, and zip code.
    - What is the coefficient on bedrooms? How does it compare to the other models? Is it statistically significant?
        - The coefficient on bedrooms is $-4.471 * 10^4$ which is greater than the previous model. It is statistically significant because the p-value is less than 0.05.
    - Suppose we wanted to use this model to make a casual statement about the effect of bedrooms. Write a full sentence about the assumption we would have to make.
        - We would have to assume that if we were to keep the square footage of living space and zip code constant, then the number of bedrooms would have a causal effect on the price of the house.


In [10]:
dummies = pd.get_dummies(df['zipcode'], drop_first=True, dtype=int)
X = df[['bedrooms', 'sqft_living']].join(dummies)
Y = df['price']

X_full = sm.add_constant(X)

model = sm.OLS(Y, X_full)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.738
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     856.1
Date:                Tue, 21 Nov 2023   Prob (F-statistic):               0.00
Time:                        22:46:18   Log-Likelihood:            -2.9312e+05
No. Observations:               21613   AIC:                         5.864e+05
Df Residuals:                   21541   BIC:                         5.870e+05
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -9.778e+04    1.1e+04     -8.878      0.000   -1.19e+05   -7.62e+04
bedrooms    -4.471e+04   1715.670    -26.058      0.000   -4.81e+04   -4.13e+04
sqft_living   278.7524      1.855    150.311      0.000     275.117     282.387
98002        2.705e+04   1.66e+04      1.629      0.103   -5503.018    5.96e+04
98003        4294.6204    1.5e+04      0.287      0.774    -2.5e+04    3.36e+04
98004        8.151e+05   1.46e+04     56.004      0.000    7.87e+05    8.44e+05
98005        3.395e+05   1.76e+04     19.292      0.000    3.05e+05    3.74e+05
98006        3.241e+05   1.31e+04     24.770      0.000    2.98e+05     3.5e+05
98007        2.772e+05   1.87e+04     14.837      0.000    2.41e+05    3.14e+05
98008        3.156e+05   1.49e+04     21.130      0.000    2.86e+05    3.45e+05
98010        7.267e+04   2.13e+04      3.419      0.001     3.1e+04    1.14e+05
98011        1.187e+05   1.67e+04      7.101      0.000     8.6e+04    1.51e+05
98014        9.713e+04   1.96e+04      4.956      0.000    5.87e+04    1.36e+05
98019        6.759e+04   1.69e+04      4.009      0.000    3.45e+04    1.01e+05
98022        4.353e+04   1.58e+04      2.758      0.006    1.26e+04    7.45e+04
98023       -1.702e+04    1.3e+04     -1.310      0.190   -4.25e+04    8438.259
98024        1.741e+05   2.31e+04      7.524      0.000    1.29e+05    2.19e+05
98027        1.719e+05   1.36e+04     12.651      0.000    1.45e+05    1.99e+05
98028        1.241e+05   1.49e+04      8.315      0.000    9.49e+04    1.53e+05
98029        2.269e+05   1.44e+04     15.717      0.000    1.99e+05    2.55e+05
98030        3386.9659   1.54e+04      0.220      0.826   -2.67e+04    3.35e+04
98031        1.569e+04   1.51e+04      1.042      0.298   -1.38e+04    4.52e+04
98032        1.623e+04   1.95e+04      0.832      0.406    -2.2e+04    5.45e+04
98033        3.969e+05   1.34e+04     29.560      0.000    3.71e+05    4.23e+05
98034        2.229e+05   1.28e+04     17.476      0.000    1.98e+05    2.48e+05
98038        1.956e+04   1.26e+04      1.556      0.120   -5071.363    4.42e+04
98039         1.38e+06   2.85e+04     48.362      0.000    1.32e+06    1.44e+06
98040        6.061e+05   1.51e+04     40.251      0.000    5.77e+05    6.36e+05
98042        8754.5989   1.27e+04      0.687      0.492   -1.62e+04    3.37e+04
98045        1.064e+05   1.61e+04      6.622      0.000    7.49e+04    1.38e+05
98052        2.458e+05   1.26e+04     19.440      0.000    2.21e+05    2.71e+05
98053         1.89e+05   1.37e+04     13.804      0.000    1.62e+05    2.16e+05
98055        4.869e+04   1.52e+04      3.211      0.001     1.9e+04    7.84e+04
98056          1.1e+05   1.36e+04      8.089      0.000    8.34e+04    1.37e+05
98058        3.755e+04   1.33e+04      2.834      0.005    1.16e+04    6.35e+04
98059        8.752e+04   1.32e+04      6.63

- g) Run one more model to evaluate the effect of bedrooms on price, picking some other variable(s) for controls. What variables did you include? Write the full estimating equation, and include a screenshot of your results. What coefficient for bedrooms do you find?
    - I included square footage of living space, bathrooms, and number of floors as controls, while still keeping bedrooms. The full estimating equation is: 
        - $\text{Price} = 7.467 * 10^4 - 5.785 * 10^4 * \text{Bedrooms} + 309.3932 * \text{Sqft\_living} + 7853.5216 * \text{Bathrooms} + 200.4972 * \text{Floors}$
    - The coefficient for bedrooms is $-5.785 * 10^4$ which is close to the coefficient in the previous two models.

In [11]:
X = df[['bedrooms', 'sqft_living', 'bathrooms', 'floors']]


Y = df['price']

X_full = sm.add_constant(X)

model = sm.OLS(Y, X_full)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     5554.
Date:                Tue, 21 Nov 2023   Prob (F-statistic):               0.00
Time:                        22:46:18   Log-Likelihood:            -2.9996e+05
No. Observations:               21613   AIC:                         5.999e+05
Df Residuals:                   21608   BIC:                         6.000e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        7.467e+04   7679.122      9.724      0.000    5.96e+04    8.97e+04
bedrooms    -5.785e+04   2347.323    -24.644      0.000   -6.24e+04   -5.32e+04
sqft_living   309.3932      3.087    100.228      0.000     303.343     315.444
bathrooms    7853.5216   3814.223      2.059      0.040     377.363    1.53e+04
floors        200.4972   3775.505      0.053      0.958   -7199.772    7600.766
==============================================================================
Omnibus:                    14450.413   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           494760.943
Skew:                           2.739   Prob(JB):                         0.00
Kurtosis:                      25.790   Cond. No.                     1.04e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""